***Load important Libraries***

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 100000)
from IPython.display import display
import os
import eli5

# sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

**helper for displaying the DataFrame**


helper for displaying a DataFrame in text classification is useful for visualizing the dataset, aiding in data cleaning, conducting exploratory data analysis (EDA), feature engineering, evaluating model performance, and creating clear documentation and reports.

In [2]:
# a helper for displaying the DataFrame
def highlight_col(x, df):
    # set by condition
    pos_mask = (df['label'] == 'pos')
    neg_mask = (df['label'] == 'neg')
    x = pd.DataFrame('', index=df.index, columns=df.columns)
    x.loc[pos_mask] = 'background-color: #e6ffe6'
    x.loc[neg_mask] = 'background-color: #ffe6e6'
    return x    

**Load the Dataset(the train data)**

In [4]:
# read tsv files for train
POS_PATH = os.path.join(os.getcwd(), 'Data', '/kaggle/input/arabic-sentiment-twitter-corpus/train_Arabic_tweets_positive_20190413.tsv')
NEG_PATH = os.path.join(os.getcwd(), 'Data', '/kaggle/input/arabic-sentiment-twitter-corpus/train_Arabic_tweets_negative_20190413.tsv')

df_pos = pd.read_csv(POS_PATH, sep='\t', header=None)
df_neg = pd.read_csv(NEG_PATH, sep='\t', header=None)

# Concate both
df_train = pd.concat([df_pos, df_neg], ignore_index=True)
df_train.columns = ['label', 'tweet']

df_train.head()

,label,tweet
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لله، لا تبحثوا فينا عن قوة، إننا مكسورون، القوة التي…
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال في روحك أماالمنبهرون بالمظا…
2,pos,من الخير نفسه 💛
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترضى بغير القمه مجرد ساعات لاستعادة الصداره💛💙 الوصول إلى القمه مهارة ت…
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المسيار ..! . ترى كانوا يشجعون ريال مدريد ضد النصر 🤣


In [5]:
df_train.tail()

,label,tweet
45270,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوكب بعدا عن الشمس يبلغ قطره كم لو كان كرة مجوفة لأستوعب…
45271,neg,احسدك على الايم 💔
45272,neg,لأول مرة ما بنكون سوا 💔
45273,neg,بقله ليش يا واطي 🤔
45274,neg,قد طال صبري في النوى إذ تركتني كئيبا ؛ غريبا باكيا متوجعا 😔 ف يامهجتي ياروح قلبي ياراحتي آغثني ف قلبي كاد أن…


In [7]:
# No need more for them after concatenating
del df_pos, df_neg

In [8]:
# See Highlited DF with my custom function
df_tmp = df_train.sample(5)
df_tmp.style.apply(lambda x: highlight_col(x, df_tmp), axis=None)

,label,tweet
38635,neg,هه مش ينقص .. ت انعدم نهائيا 😣
6898,pos,انا مقلعش البنطلون غير في سياق الدراسه 😂
27981,neg,#الاتحاد_النصر الوضع بختصار 😏
44771,neg,Circus النسخة الكورية !! اوكيه عبالي ماراح يقربون من الاغاني اليابانية بس طبعا راضية عشانها تحمس مرة بموت 😭
4874,pos,#يسقط_حكم_تميم البلد بازط يا جدعان متعرفش الراجل من اللي مش راجل من المقنس والمش مقنس بلد مضروبه بخلاط الحجه فواكه…


**load the dataset(the test data)**

In [9]:
# read tsv files for train
TEST_POS_PATH = os.path.join(os.getcwd(), 'Data', '/kaggle/input/arabic-sentiment-twitter-corpus/test_Arabic_tweets_positive_20190413.tsv')
TEST_NEG_PATH = os.path.join(os.getcwd(), 'Data', '/kaggle/input/arabic-sentiment-twitter-corpus/test_Arabic_tweets_negative_20190413.tsv')

df_test_pos = pd.read_csv(TEST_POS_PATH, sep='\t', header=None)
df_test_neg = pd.read_csv(TEST_NEG_PATH, sep='\t', header=None)

# Concate both
df_test = pd.concat([df_test_pos, df_test_neg], ignore_index=True)
df_test.columns = ['label', 'tweet']

df_test.head()

,label,tweet
0,pos,#الهلال_الاهلي فوز هلالي مهم الحمد لله 💙 زوران كان بيسلم المباراة بعد تبديل كارييو بإنتظار الإتحاد بكرة يارب يار…
1,pos,صباحك خيرات ومسرات 🌸
2,pos,"#تأمل قال الله ﷻ :- _*​﴿بواد غير ذي زرع ﴾*_ 💫💫 ✍ "" ~ومع ذلك هتف بالدعاء ﴿وارزقهم من الثمرات ﴾ مهماكانت ظرو…"
3,pos,😂😂 يا جدعان الرجاله اللي فوق ال دول خطر ع تويتر وربنا 😂مش اسلوب كل يومين يدخلي واحد قد جدي علشان يشقطني 😒 😹و عند…
4,pos,رساله صباحيه : 💛 اللهم اسألك التوفيق في جميع امورنا واكتب لنا الفردوس نحن ووالدينا وجميع موتى المسلمين برحمتك يا ارحم الراحمين


In [10]:
# No need more for them after concatenating
del df_test_pos, df_test_neg

In [12]:
# See Highlited DF with my custom function
df_tmp = df_test.sample(5)
df_tmp.style.apply(lambda x: highlight_col(x, df_tmp), axis=None)

,label,tweet
7384,neg,المهم مشكور ع جهودك حاولت اضحك بس ماش والله ☹
7236,neg,اذا الفيفا عارف هالشي غلط مش المفروض يتصرف ويتخذ اجراءه ولا الاتحاد السعودي اقوى منه ولا هاي الشوشرا…
4509,pos,طبعة الروج 💋 هي العلامة اللي تعرف بها البنت نقابها اذا مقلوب او لا صحيح هالكلام ي بنات ؟! 🌚😂
5203,pos,سورى بجد بس انتى قمر فشخ ❤ — 💖💖
10113,neg,انا خايف من النصر وحظه السييء 💔


**Baseline model (using pipeline)**

In [13]:
# Vectorizing and then model
vect = CountVectorizer(max_features=15000, encoding='utf-8')
clf = LogisticRegression(max_iter=10000)

# combine them to a pipeline
pipe = Pipeline(steps=[
        ('vectorizer', vect),
        ('classifier', clf)
    ])

# fitting to train data
pipe.fit(df_train['tweet'], df_train['label'])

Pipeline(steps=[('vectorizer', CountVectorizer(max_features=15000)),
                ('classifier', LogisticRegression(max_iter=10000))])

In [14]:
df_train['label'].value_counts()   # balanced dataset

label
pos    22761
neg    22514
Name: count, dtype: int64

In [15]:
# predict on test
y_pred_test = pipe.predict(df_test['tweet'])
print(f'Accuracy is {accuracy_score(df_test["label"], y_pred_test) * 100:.3f} %')

Accuracy is 77.274 %


**Try on some tweets**

In [16]:
for _, row in df_test.sample(5).iterrows():
    print(f"True label: {row['label']}")
    display(eli5.show_prediction(clf, row['tweet'], vec=vect, top=10, feature_names=vect.get_feature_names_out()))
    print("--"*50)

# Green: Features that support the predicted class. These features increase the likelihood of the model predicting the shown class.
# Red: Features that support the opposite class. These features decrease the likelihood of the model predicting the shown class.

True label: neg


----------------------------------------------------------------------------------------------------
True label: neg


----------------------------------------------------------------------------------------------------
True label: pos


----------------------------------------------------------------------------------------------------
True label: pos


----------------------------------------------------------------------------------------------------
True label: neg


----------------------------------------------------------------------------------------------------


**Using TFIDF and SVC in one pipeline**

In [17]:
vect = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5), min_df=0.01, max_df=0.5, max_features=10000)
clf = LinearSVC()
pipe_tfidf = Pipeline(steps=[
        ('vectorizer', vect),
        ('classifier', clf)
    ])
pipe_tfidf.fit(df_train['tweet'], df_train['label'])

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char_wb', max_df=0.5,
                                 max_features=10000, min_df=0.01,
                                 ngram_range=(3, 5))),
                ('classifier', LinearSVC())])

In [18]:
# predict on test
y_pred_test = pipe_tfidf.predict(df_test['tweet'])
print(f'Accuracy is {accuracy_score(df_test["label"], y_pred_test) * 100:.3f} %')

Accuracy is 83.811 %


In [19]:
for _, row in df_test.sample(5).iterrows():
    print(f"True label: {row['label']}")
    display(eli5.show_prediction(clf, row['tweet'], vec=vect, top=10, feature_names=vect.get_feature_names_out()))
    print("--"*50)

True label: neg


----------------------------------------------------------------------------------------------------
True label: neg


----------------------------------------------------------------------------------------------------
True label: neg


----------------------------------------------------------------------------------------------------
True label: neg


----------------------------------------------------------------------------------------------------
True label: neg


----------------------------------------------------------------------------------------------------


In [20]:
import joblib
joblib.dump(pipe_tfidf, 'model.pkl')

['model.pkl']